In [213]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
import pandas as pd
import numpy as np
np.random.seed(0)

In [243]:
data = pd.read_csv("Cristano_Ronaldo_Final_v1/data.csv")
data = data.drop(data.columns[0],axis=1)
# data.head()

# purged features:
data = data.drop(['shot_id_number'],axis=1)
data = data.drop(['date_of_game'],axis=1)
data = data.drop(['team_id'],axis=1)
# data = data.drop(['type_of_shot'],axis=1)
data = data.drop(['game_season'], axis=1)
# data = data[data['shot_id_number'].notnull()]

data = pd.get_dummies(data)

# data = data.drop(["type_of_shot_shot - 37","home/away_MANU @ NOK","shot_basics_Right Corner","home/away_MANU vs. NOP","lat/lng_40.324211, -111.674849","type_of_shot_shot - 33",
# "home/away_MANU vs. VAN",
# "type_of_combined_shot_shot - 2",
# "shot_basics_Left Corner",
# "type_of_shot_shot - 34",
# "home/away_MANU vs. CHH",
# "home/away_MANU @ VAN",
# "lat/lng_49.250068, -123.114646",
# "lat/lng_30.055498, -89.960838",
# "lat/lng_35.205878, -80.841194",
# "home/away_MANU @ NOP",
# "home/away_MANU @ CHH",
# "home/away_MANU vs. PHO",
# "range_of_shot_Back Court Shot",
# "area_of_shot_Mid Ground(MG)",
# "game_season_2013-14",
# "home/away_MANU vs. SAN",
# "lat/lng_33.513157, -112.082793",
# "home/away_MANU @ PHO",
# "home/away_MANU @ BKN",
# "lat/lng_40.623199, -73.951223",
# "shot_basics_Mid Ground Line",
# "home/away_MANU @ UTH",
# "home/away_MANU vs. BKN"],axis=1)

In [244]:
#######LOCAL TRAIN-VALIDATION SPLIT########
#Rescale all feature values:
mm_scaler = preprocessing.MinMaxScaler()

#global train-test split:
train = data[data['is_goal'].notnull()]
y = train['is_goal']
test = data[data['is_goal'].isnull()]

# local training / validation performance:
train['local_train']=np.random.uniform(0,1,len(train))<=0.85
local_train, local_validation = train[train['local_train']==True], train[train['local_train']==False]
y_local_train = local_train['is_goal']
y_local_validation=local_validation['is_goal']
local_train=local_train.drop(['is_goal'],axis=1)
local_validation =local_validation.drop(['is_goal'],axis=1)

features = local_train.columns[:-1]
print(features)
local_train = local_train[features]
local_validation = local_validation[features]
print("local train size:", len(local_train))
print("local validation size:", len(local_validation))
print("test size:", len(test))

# preprocessing local-train+validation:
imp= SimpleImputer(missing_values=np.nan,strategy='mean')
imp = imp.fit(local_train)
local_train_modified = imp.transform(local_train)
imp = imp.fit(local_validation)
local_validation_modified = imp.transform(local_validation)
# local_train_modified = mm_scaler.fit_transform(local_train_modified)
# local_validation_modified = mm_scaler.fit_transform(local_train_modified)

#RF classifier for train-validation perf:
clf = RandomForestClassifier(n_jobs=2, n_estimators=500, random_state=0)
clf.fit(local_train_modified, y_local_train)
y_validation_pred = clf.predict(local_validation_modified)
count_match = 0
count_error = 0

print("type,",type(y_validation_pred),type(y_local_validation))
assert(len(y_validation_pred)==len(y_local_validation))
validation_gtruth=np.asarray(y_local_validation)
for i in range(len(y_local_validation)):
    if (int(y_validation_pred[i])==int(validation_gtruth[i])):
        count_match+=1
    else:
        count_error+=1
validation_accuracy = count_match/(count_match+count_error)*100.0
print("validation a/c:", validation_accuracy)
features_imp = pd.DataFrame(clf.feature_importances_, index=features,columns=['importance']).sort_values('importance', ascending=False)
print(features_imp)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Index(['match_event_id', 'location_x', 'location_y', 'remaining_min',
       'power_of_shot', 'knockout_match', 'remaining_sec', 'distance_of_shot',
       'match_id', 'remaining_min.1',
       ...
       'type_of_shot_shot - 6', 'type_of_shot_shot - 7',
       'type_of_shot_shot - 8', 'type_of_shot_shot - 9',
       'type_of_combined_shot_shot - 0', 'type_of_combined_shot_shot - 1',
       'type_of_combined_shot_shot - 2', 'type_of_combined_shot_shot - 3',
       'type_of_combined_shot_shot - 4', 'type_of_combined_shot_shot - 5'],
      dtype='object', length=208)
local train size: 20824
local validation size: 3605
test size: 6268
type, <class 'numpy.ndarray'> <class 'pandas.core.series.Series'>
validation a/c: 62.08044382801664
                                    importance
match_id                              0.063251
match_event_id                        0.062059
remaining_sec.1                       0.057779
remaining_sec                         0.055349
location_y               

In [174]:
#######ACTUAL TRAIN-TEST PERDICTION########
y_train = train['is_goal']
train=train.drop(['is_goal'],axis=1)
test =test.drop(['is_goal'],axis=1)
features = local_train.columns[:-1]
train = train[features]
test = test[features]
print("Train size:", len(train))
print("Test size:", len(test))

# preprocessing local-train+validation:
imp= SimpleImputer(missing_values=np.nan,strategy='mean')
imp = imp.fit(train)
train_modified = imp.transform(train)
imp = imp.fit(test)
test_modified = imp.transform(test)

#RF classifier for train-validation perf:
clf2 = RandomForestClassifier(n_jobs=2, n_estimators=100, random_state=0)
clf2.fit(train_modified, y_train)
# y_test_pred = clf2.predict(test_modified)
p = clf2.predict_proba(test_modified)
prediction = []
for x,y in p:
    prediction.append(y)
count_match = 0
count_error = 0

# assert(len(y_test_pred)==len(y_local_validation))
# validation_gtruth=np.asarray(y_local_validation)
# for i in range(len(y_local_validation)):
#     if (int(y_validation_pred[i])==int(validation_gtruth[i])):
#         count_match+=1
#     else:
#         count_error+=1
# validation_accuracy = count_match/(count_match+count_error)*100.0
# print("validation a/c:", validation_accuracy)

Train size: 24429
Test size: 6268


In [177]:
#write outputs:
# shot_arr = np.asarray(test['shot_id_number'])
test_rows = data.index[data.is_goal.isnull()]
count = 0
f = open("submissionAP.csv","w+")
print("shot_id_number,is_goal", file=f)
# print("shot_id_number,is_goal", file=f)
for i in range(len(test_rows)):
    print(str(int(test_rows[i]+1))+","+str(prediction[i]), file=f)
    count+=1
print(count)
f.close()

6268


In [124]:
print(validation_gtruth)

[1. 1. 1. ... 0. 0. 0.]


In [142]:
print(len(test))

6268


In [185]:
# list(zip(train[features],clf2.feature_importances_))
features_imp = pd.DataFrame(clf.feature_importances_, index=features,columns=['importance']).sort_values('importance', ascending=False)
print(features_imp)

                                     importance
match_event_id                     4.062614e-02
match_id                           4.043662e-02
distance_of_shot.1                 3.886515e-02
location_y                         3.806000e-02
remaining_sec                      3.752482e-02
team_id                            3.705409e-02
location_x                         3.665255e-02
game_season_1996-97                3.538272e-02
power_of_shot.1                    3.279888e-02
remaining_min                      2.895486e-02
distance_of_shot                   2.892361e-02
knockout_match.1                   2.569584e-02
remaining_sec.1                    1.979802e-02
power_of_shot                      1.720950e-02
shot_basics_Goal Line              8.132400e-03
type_of_combined_shot_shot - 4     7.947864e-03
lat/lng_43.062206, -87.944754      7.935073e-03
area_of_shot_Left Side Center(LC)  6.941969e-03
knockout_match                     6.820156e-03
type_of_combined_shot_shot - 2     6.660

In [162]:

print(prediction)
    

[0.38, 0.48, 0.51, 0.69, 0.37, 0.4, 0.59, 0.57, 0.51, 0.62, 0.34, 0.41, 0.41, 0.34, 0.33, 0.3, 0.41, 0.34, 0.37, 0.3, 0.36, 0.37, 0.31, 0.49, 0.43, 0.46, 0.58, 0.33, 0.47, 0.32, 0.86, 0.34, 0.52, 0.54, 0.37, 0.4, 0.4, 0.48, 0.4, 0.44, 0.43, 0.61, 0.58, 0.47, 0.71, 0.41, 0.44, 0.38, 0.43, 0.39, 0.44, 0.51, 0.8, 0.66, 0.32, 0.51, 0.31, 0.4, 0.41, 0.7, 0.42, 0.44, 0.69, 0.36, 0.35, 0.41, 0.28, 0.21, 0.32, 0.61, 0.4, 0.28, 0.41, 0.32, 0.47, 0.65, 0.7, 0.47, 0.41, 0.41, 0.42, 0.31, 0.32, 0.28, 0.57, 0.69, 0.34, 0.42, 0.39, 0.4, 0.43, 0.4, 0.36, 0.36, 0.24, 0.42, 0.42, 0.56, 0.36, 0.27, 0.41, 0.77, 0.17, 0.4, 0.39, 0.69, 0.31, 0.94, 0.64, 0.5, 0.42, 0.51, 0.62, 0.32, 0.33, 0.37, 0.31, 0.48, 0.47, 0.38, 0.82, 0.8, 0.5, 0.54, 0.72, 0.38, 0.81, 0.69, 0.5, 0.49, 0.53, 0.36, 0.23, 0.32, 0.3, 0.42, 0.56, 0.38, 0.43, 0.47, 0.38, 0.47, 0.42, 0.34, 0.52, 0.39, 0.44, 0.37, 0.35, 0.34, 0.42, 0.48, 0.35, 0.36, 0.3, 0.3, 0.32, 0.42, 0.34, 0.32, 0.32, 0.36, 0.51, 0.42, 0.82, 0.26, 0.35, 0.4, 0.3, 0.89, 0.

In [150]:
print(test_rows)

Int64Index([    0,     7,    16,    19,    21,    32,    33,    34,    35,
               36,
            ...
            30659, 30664, 30668, 30679, 30680, 30681, 30682, 30686, 30687,
            30693],
           dtype='int64', length=6268)
